# Setup

In [1]:
# Import necessary modules and libraries
from abc import ABC, abstractmethod
import base64
import requests
import pandas as pd

# Define FlexStackAPIHub

In [2]:
class FlexStack(ABC):
    @abstractmethod
    def generate(self):
        """
        Generate method to be implemented by subclasses. This method should handle the generation process specific to the subclass.
        """
        pass


class FlexStackAPIHub(FlexStack):
    """
    FlexStackAPIHub is a class designed to interact with the FlexStack API Marketplace.
    It allows for user authentication, listing available APIs, retrieving API usage guides,
    adding new APIs to the marketplace, and making API calls.

    Attributes:
        url (str): The base URL for the FlexStack API Marketplace.
        token (str): Authentication token for the API Marketplace.
        apis (pd.DataFrame): A DataFrame containing the list of available APIs.
    """

    def __init__(self, url: str, token: str) -> None:
        """
        Initializes the FlexStackAPIHub class with a base URL and an authentication token.

        Args:
            url (str): The base URL for the FlexStack API Marketplace.
            token (str): Authentication token for the API Marketplace.
        """
        self.url = url
        self.token = token
        self.apis = self.list_api()

    @classmethod
    def login(cls, username: str, password: str) -> 'FlexStackAPIHub':
        """
        Authenticates a user with the FlexStack API Marketplace and returns an instance of the class.

        Args:
            username (str): The username for login.
            password (str): The password for login.

        Returns:
            FlexStackAPIHub: An instance of the class with an authentication token.
        """
        def basic_auth(username: str, password: str) -> str:
            token = base64.b64encode(f"{username}:{password}".encode('utf-8')).decode("ascii")
            return f'Basic {token}'

        base_url = 'https://api.joinparrot.ai/v1' 

        url = f'{base_url}/user/login'

        headers = {'Authorization': basic_auth(username, password)}
        response = requests.post(url, headers=headers)
        if response.status_code == 200:
            print("[INFO] Login success")
        else:
            print(f"[ERROR] Login failed. {response.text}")
        token = response.json()["data"]["access_token"]
        return cls(base_url, token)

    def list_api(self) -> pd.DataFrame:
        """
        Retrieves a list of available APIs from the FlexStack API Marketplace.

        Returns:
            pd.DataFrame: A DataFrame containing the list of available APIs.
        """
        api_list_url = f'{self.url}/api-market/list-api'
        headers = {'Authorization': f'Bearer {self.token}'}
        response = requests.get(api_list_url, headers=headers)
        if response.status_code == 200:
            return pd.DataFrame(response.json()['data'])
        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")

    def guide_usage(self, api_id: int) -> pd.Series:
        """
        Retrieves the usage guide for a specific API by its ID.

        Args:
            api_id (int): The ID of the API.

        Returns:
            pd.Series: A Series containing the usage information of the specified API.
        """
        self.apis = self.list_api()
        api_info = self.apis[self.apis['id'] == api_id].iloc[0]

        api_name = api_info['name']
        endpoint = api_info.api_path

        examples = {}
        if api_info.method == "POST" and "application" in api_info.body_type:
            examples = {"body": {}}
            for param in api_info.params:
                examples['body'].update({param['key']: ""})
        if api_info.method == "GET":
            examples = {"query": {}}
            for param in api_info.params:
                examples['query'].update({param['key']: ""})
                
        print("> Usage Guide")
        print(f"\t- API Name: {api_name}")
        print(f"\t- Endpoint: {endpoint}")
        print("\t- Parameters:")
        for param in api_info.params:
            print(f"\t\t- {param['key']}: {param['desc']}")
        print(f"\t- Examples: \n\t{examples}")

        return api_info

    def add_api(self, type: str, data: dict) -> dict:
        """
        Adds a new API to the FlexStack API Marketplace.

        Args:
            type (str): The type of API being added (e.g., "FORM", "POSTMAN_COLLECTION", "OPENAPI").
            data (dict): The data required to add the new API.

        Returns:
            dict: The response from the API Marketplace after adding the new API.
        """
        if type not in ["FORM", "POSTMAN_COLLECTION", "OPENAPI"]:
            raise ValueError(f"Error: Unsupported API type '{type}'.")
        add_api_url = f'{self.url}/api-market/add'
        headers = {'Authorization': f'Bearer {self.token}'}
        response = requests.post(add_api_url, json=data, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")

    def generate(self, endpoint: str, data: dict) -> dict:
        """
        Makes a call to a specified API endpoint with provided data.

        Args:
            endpoint (str): The endpoint of the API to be called.
            data (dict): The data to be sent in the API call.

        Returns:
            dict: The response from the API call.
        """
        headers = {'Authorization': f'Bearer {self.token}'}
        response = requests.post(endpoint, json=data, headers=headers)
        return response.json()


# API Market

## Login

In [5]:
username = "<your-username>"
password = "<your-password>"

# Login
flexstack_apihub = FlexStackAPIHub.login(username, password)

[INFO] Login success


## Discovery

In [6]:
flexstack_apihub.list_api() # View list API 

,id,api_path,name,desc,method,fee,is_active,notes,params,body_type
0,110,https://api.joinparrot.ai/v1/api-market/use/88...,Get Fitness feeds,Fitness feeds,GET,0.0,True,None,[],application/rss+xml
1,108,https://api.joinparrot.ai/v1/api-market/use/ed...,Get Coffee feeds,Coffee feeds,GET,0.0,True,None,[],application/rss+xml
2,106,https://api.joinparrot.ai/v1/api-market/use/76...,Get Health feeds,Health feeds,GET,0.0,True,None,[],application/rss+xml
3,105,https://api.joinparrot.ai/v1/api-market/use/85...,Get Food feeds,Food feeds,GET,0.0,True,None,[],application/rss+xml
4,102,https://api.joinparrot.ai/v1/api-market/use/63...,Get News feeds,News feeds,GET,0.0,True,None,[],application/rss+xml
5,101,https://api.joinparrot.ai/v1/api-market/use/91...,Get Writing feeds,Writing feeds,GET,0.0,True,None,[],application/rss+xml
6,100,https://api.joinparrot.ai/v1/api-market/use/bb...,Get Real Estate feeds,Real Estate feeds,GET,0.0,True,None,[],application/rss+xml
7,98,https://api.joinparrot.ai/v1/api-market/use/d1...,Get Photography feeds,Photography feeds,GET,0.0,True,None,[],application/rss+xml
8,97,https://api.joinparrot.ai/v1/api-market/use/25...,Get Law feeds,Law feeds,GET,0.0,True,None,[],application/rss+xml
9,96,https://api.joinparrot.ai/v1/api-market/use/f2...,Get Marketing feeds,Marketing feeds,GET,0.0,True,None,[],application/rss+xml


# Get API Feeds with FlexStack

## Get API Info and Guide

### 1. API GET Music Feeds

In [7]:
api_music = flexstack_apihub.guide_usage(api_id=59) 

> Usage Guide
	- API Name: Get Music feeds
	- Endpoint: https://api.joinparrot.ai/v1/api-market/use/67f69b98b5be438289e7148e696e4a19
	- Parameters:
	- Examples: 
	{'query': {}}


### 2. API GET Tech Feeds

In [8]:
api_music = flexstack_apihub.guide_usage(api_id=90)

> Usage Guide
	- API Name: Get Tech feeds
	- Endpoint: https://api.joinparrot.ai/v1/api-market/use/72c482f543794bfc923a10fa975b20da
	- Parameters:
	- Examples: 
	{'query': {}}


## Usage

### Get result

In [12]:
data={} 

endpoint = "https://api.joinparrot.ai/v1/api-market/use/67f69b98b5be438289e7148e696e4a19" # Endpoint  of the API you want to use

resp = flexstack_apihub.generate(endpoint, data)                                          # Call the API to generate a response

print(resp)

{'data': '<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"\n\txmlns:content="http://purl.org/rss/1.0/modules/content/"\n\txmlns:wfw="http://wellformedweb.org/CommentAPI/"\n\txmlns:dc="http://purl.org/dc/elements/1.1/"\n\txmlns:atom="http://www.w3.org/2005/Atom"\n\txmlns:sy="http://purl.org/rss/1.0/modules/syndication/"\n\txmlns:slash="http://purl.org/rss/1.0/modules/slash/"\n\t\n\txmlns:georss="http://www.georss.org/georss"\n\txmlns:geo="http://www.w3.org/2003/01/geo/wgs84_pos#"\n\t>\n\n<channel>\n\t<title>Billboard</title>\n\t<atom:link href="https://www.billboard.com/feed/" rel="self" type="application/rss+xml" />\n\t<link>https://www.billboard.com</link>\n\t<description>Music Charts, News, Photos &#38; Video</description>\n\t<lastBuildDate>Fri, 29 Mar 2024 13:30:35 +0000</lastBuildDate>\n\t<language>en-US</language>\n\t<sy:updatePeriod>\n\thourly\t</sy:updatePeriod>\n\t<sy:updateFrequency>\n\t1\t</sy:updateFrequency>\n\t<generator>https://wordpress.org/?v=6.4.3</generator>\n

### Get data feeds

In [14]:
data_content = resp['data'] # Get data from response
print(data_content)

<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"
	xmlns:content="http://purl.org/rss/1.0/modules/content/"
	xmlns:wfw="http://wellformedweb.org/CommentAPI/"
	xmlns:dc="http://purl.org/dc/elements/1.1/"
	xmlns:atom="http://www.w3.org/2005/Atom"
	xmlns:sy="http://purl.org/rss/1.0/modules/syndication/"
	xmlns:slash="http://purl.org/rss/1.0/modules/slash/"
	
	xmlns:georss="http://www.georss.org/georss"
	xmlns:geo="http://www.w3.org/2003/01/geo/wgs84_pos#"
	>

<channel>
	<title>Billboard</title>
	<atom:link href="https://www.billboard.com/feed/" rel="self" type="application/rss+xml" />
	<link>https://www.billboard.com</link>
	<description>Music Charts, News, Photos &#38; Video</description>
	<lastBuildDate>Fri, 29 Mar 2024 13:30:35 +0000</lastBuildDate>
	<language>en-US</language>
	<sy:updatePeriod>
	hourly	</sy:updatePeriod>
	<sy:updateFrequency>
	1	</sy:updateFrequency>
	<generator>https://wordpress.org/?v=6.4.3</generator>
	<item>
		<title>BMI Members to Receive $100M Bonus From 